In [1]:
import os

%pwd

'd:\\work\\cifar100classifier-deep-learning\\research'

In [2]:
os.chdir('../')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [3]:
import sys

sys.path.append('src')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [ ]:
import tensorflow as tf

# model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    testset_file: Path
    all_params: dict
    num_classes: int
    params_image_size: list
    params_batch_size: int

In [4]:
import tensorflow as tf

from CNNClassifier.constants import *
from CNNClassifier.utils.utilities import read_yaml, create_directory, save_json

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= Path(self.config.training.trained_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir),
            testset_file=Path(self.config.data_ingestion.testset_file),
            all_params=self.params,
            num_classes=self.params.CLASSES,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [9]:
import pickle

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def unpickle(self, file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='latin1')
        return dict
    
    def extract_data(self, dicts, num_classes):
        images = dicts['data']
        n = len(images)
        images = images.reshape(n, 3, 32, 32).transpose(0, 2, 3, 1)
        # take coarse and fine labels of the images
        c_labels = dicts['coarse_labels']
        f_labels = dicts['fine_labels']
        # transform label to categorical
        c_labels = tf.keras.utils.to_categorical(c_labels, num_classes)
        f_labels = tf.keras.utils.to_categorical(f_labels, num_classes)
        # normalize data
        # images = images/255
        return images, f_labels, c_labels
    
    def _valid_generator(self):
        test_set = self.unpickle(os.path.join(self.config.training_data, self.config.testset_file))
        x_test, y_test, _ = self.extract_data(test_set, self.config.num_classes)

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )
        
        self.test_generator = test_datagenerator.flow(x_test, y_test, batch_size=1)
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.test_generator)
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2023-10-22 10:41:34,168: INFO: utilities]: yaml file: configs\config.yaml loaded successfully
[2023-10-22 10:41:34,173: INFO: utilities]: yaml file: params.yaml loaded successfully
[2023-10-22 10:41:34,176: INFO: utilities]: create directory at: artifacts
10000/10000 [==============================] - 179s 18ms/step - loss: 2.6768 - accuracy: 0.3276
[2023-10-22 10:44:34,045: INFO: utilities]: json file saved at: scores.json
